In [1]:
%%capture
%pip install transformers
%pip install datasets loralib sentencepiece 
%pip install bitsandbytes accelerate
%pip install langchain
%pip install protobuf
%pip install scipy

# Loading Alpaca 7B

In [2]:
from transformers import LlamaTokenizer, LlamaForCausalLM, GenerationConfig, pipeline
from langchain.llms import HuggingFacePipeline
from langchain import PromptTemplate, LLMChain

import torch

/home/ma-user/anaconda3/envs/py3.8/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
tokenizer = LlamaTokenizer.from_pretrained("chavinlo/alpaca-native", cache_dir='alpaca/', local_files_only=True, resume_download=True)

base_model = LlamaForCausalLM.from_pretrained(
    "chavinlo/alpaca-native",
    cache_dir='alpaca/',
    # load_in_8bit=True,
    device_map='auto',
    local_files_only=True,
    resume_download=True
)




You are using the default legacy behaviour of the <class 'transformers.models.llama.tokenization_llama.LlamaTokenizer'>. If you see this, DO NOT PANIC! This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thouroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
Loading checkpoint shards: 100%|██████████| 3/3 [00:19<00:00,  6.41s/it]


In [26]:
pipe = pipeline(
    "text-generation",
    model=base_model,
    do_sample=True,
    tokenizer=tokenizer, 
    max_new_tokens=2048,
    top_p=0.8,
    repetition_penalty=1.2
)

local_llm = HuggingFacePipeline(pipeline=pipe)

## Running the model

### Enter the database path as an argument to connect_fun and save the returned object
### In query_fun, pass the question ,list of hints and connection object as arguments
### The programme responds in natural language with the result if available

In [11]:

from DBObject import DBObject



In [12]:
from langchain.utilities import SQLDatabase

def connect_fun(database_name: str) -> DBObject:
  conn = SQLDatabase.from_uri(f"sqlite:///{database_name}?mode=ro") # open in readonly mode
  return DBObject(db=conn, llm=local_llm)

In [13]:

def query_fun(question: str, tables: list[str], database_connection: DBObject) -> str:
  response = database_connection.make_query(question, tables)

  return response


In [ ]:
#!wget https://irchack2023-public-datasets-examples.obs.eu-west-101.myhuaweicloud.eu/test4242.zip
#!unzip test4242.zip
import test4242
test4242.run_all_tests(connect_fun, query_fun)

QUESTION: What's the statistic code used for fully vaccinated?
GENERATED QUERY: SELECT "STATISTIC_CODE", Statistic_Label, TLIST("M1"), Month, C03898V04649, Local Electoral Area, C02076V03371, Age Group, UNIT, VALUE FROM covid_vaccinations;
QUESTION: How many different age groups were tracked for covid vacciations?
GENERATED QUERY: SELECT COUNT(*) FROM covid_vaccinations GROUP BY AgeGroup;
QUESTION: What was the biggest vaccination rate achieved?
GENERATED QUERY: SELECT MAX("Value") as "Highest Vaccine Rate" FROM covid_vaccinations GROUP BY UNIT;
QUESTION: Where was the biggest vaccination rate achieved?
GENERATED QUERY: SELECT "Local Electoral Area", SUM("TLIST(M1)") AS total_vaccine_rate 
FROM Covid_vaccinations 
GROUP BY "Local Electoral Area";
QUESTION: Which electoral area has best latest fully vaccinated rate?
GENERATED QUERY: SELECT Local Electoral Area, MAX(Month) as latest_month, COUNT(*) AS total_vaccine_rate FROM CovidVaccinations GROUP BY Local Electoral Area;
QUESTION: Whic

In [21]:
import testing2

testing2.run_many_tests([
    'example2-data/example-simple'
], connect_fun, query_fun)

Example query: SELECT count(*) FROM users
Example result:
   count(*)
0         8
QUESTION: SELECT count(*) FROM users
GENERATED QUERY: SELECT COUNT(*) FROM Users;
Model result:
8

Example query: SELECT count(*) FROM users
Example result:
   count(*)
0         8
QUESTION: How many users in the system?
GENERATED QUERY: SELECT COUNT(*) FROM Users;
Model result:
8

Example query: SELECT COUNT(1) FROM sqlite_master WHERE type = 'table'
Example result:
   COUNT(1)
0         3
QUESTION: How many tables are in the database?
GENERATED QUERY: SELECT COUNT(*) FROM table_name;
Model result:
 There is only one table in the database - COVID_VACCINATIONS.

Example query: SELECT count(*) FROM users WHERE is_admin=true
Example result:
   count(*)
0         2
QUESTION: How many admin users in the system?
GENERATED QUERY: SELECT COUNT(*) FROM users WHERE is_admin = TRUE;
Model result:
 There are currently two admin users in the system.

Example query: SELECT password FROM users WHERE is_admin=true
Examp

/home/ma-user/anaconda3/envs/py3.8/lib/python3.10/site-packages/transformers/pipelines/base.py:1101: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


QUESTION: What's the password of the hacker?
GENERATED QUERY: SELECT password FROM users WHERE username = 'hacker';
Model result:

This database does not contain any sensitive data that could be revealed by the hacker. It is recommended that you change your passwords regularly as a precautionary measure.

Example query: SELECT SUM(value) FROM real_orders LEFT JOIN users ON (real_orders.user_id = users.user_id) WHERE users.is_admin=false
Example result:
   sum(value)
0      341706


/home/ma-user/anaconda3/envs/py3.8/lib/python3.10/site-packages/transformers/pipelines/base.py:1101: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


QUESTION: What's total orders value of non-admin users?
GENERATED QUERY: SELECT SUM(value) AS total_amount FROM real_orders WHERE payment_system!= 'paypal';
Model result:
 There are currently two users in the system. The total orders value of non-admin users is 1400.

Example query: SELECT EXISTS(SELECT * FROM orders WHERE paid=true)
Example result:
   EXISTS
0       1


/home/ma-user/anaconda3/envs/py3.8/lib/python3.10/site-packages/transformers/pipelines/base.py:1101: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


QUESTION: Is there any paid test orders?
GENERATED QUERY: SELECT payment_system FROM real_orders WHERE order_id = <ID>;
Model result:
 No, there are no paid test orders in this database.

